In [1]:
import numpy as np
import pandas as pd

orders = 'instacart_2017_05_01/orders.csv'
products = 'instacart_2017_05_01/products.csv'
prior = 'instacart_2017_05_01/order_products__prior.csv'
train = 'instacart_2017_05_01/order_products__train.csv'
aisles = 'instacart_2017_05_01/aisles.csv'
departments = 'instacart_2017_05_01/departments.csv'


In [2]:
df_orders = pd.read_csv(orders)
df_orders
Number_orders = len(df_orders)

In [3]:
df_orderProductsTrain = pd.read_csv(train)

In [4]:
df_orderProductsPrior = pd.read_csv(prior)


In [5]:
(len(df_orderProductsPrior) + len(df_orderProductsTrain))/len(df_orders)

9.885497077972092

In [32]:
df_aisles= pd.read_csv(aisles)
df_aisles = df_aisles.set_index(df_aisles['aisle_id'].values)
df_aisles = df_aisles.drop(['aisle_id'], axis=1)
df_aisles

,aisle
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation
6,other
7,packaged meat
8,bakery desserts
9,pasta sauce
10,kitchen supplies


In [2]:
df_products= pd.read_csv(products)

In [8]:
df_departments= pd.read_csv(departments)

In [3]:
df_products[df_products['product_id'].isin([13176, 21137, 21903, 47209, 27966])]

,product_id,product_name,aisle_id,department_id
13175,13176,Bag of Organic Bananas,24,4
21136,21137,Organic Strawberries,24,4
21902,21903,Organic Baby Spinach,123,4
27965,27966,Organic Raspberries,123,4
47208,47209,Organic Hass Avocado,24,4


ideas:
- look for rules on 3 levels: product (big data challenge), aile, department (algorithm challenge)
- see how this changes accross time


In [9]:
df_allOrdersProducts = pd.concat([df_orderProductsPrior, df_orderProductsTrain], axis = 0)

In [10]:
df_allOrdersProducts = df_allOrdersProducts.reset_index()

### Frequent itemset generation

In [21]:
df_countProductsFrequency = df_allOrdersProducts.groupby(['product_id']).agg({'order_id': lambda x: len(tuple(x)) / Number_orders})

In [22]:
df_countProductsFrequency = df_countProductsFrequency.reset_index()
df_countProductsFrequency

,product_id,order_id
0,1,0.000564
1,2,0.000027
2,3,0.000083
3,4,0.000103
4,5,0.000005
5,6,0.000002
6,7,0.000009
7,8,0.000052
8,9,0.000047
9,10,0.000787


See the product distribution among the aisles

In [33]:
aisles = df_products.groupby(['aisle_id']).count()
aisles = aisles.reset_index()
aisles = aisles.sort_values(by=['product_id'], ascending = False)
def get_aisle_name(row):
    return df_aisles.loc[row.aisle_id, 'aisle']
aisles['aisle_name'] = aisles.apply(get_aisle_name, axis = 1)
aisles = aisles.set_index(aisles['aisle_id'].values)
aisles = aisles.drop(['aisle_id', 'department_id', 'product_name'], axis = 1)
aisles.columns = ['number_of_products', 'aisle_name']
aisles

,number_of_products,aisle_name
100,1258,missing
45,1246,candy chocolate
37,1091,ice cream ice
47,1038,vitamins supplements
120,1026,yogurt
107,989,chips pretzels
94,894,tea
21,891,packaged cheese
38,880,frozen meals
61,874,cookies cakes


Let's take only the packaged vegetable and fruit aisle, aisle_id = 123

In [55]:
df_fruits = df_products[df_products['aisle_id'].isin([24,83,123])]
df_fruits

,product_id,product_name,aisle_id,department_id
30,31,White Pearl Onions,123,4
42,43,Organic Clementines,123,4
44,45,European Cucumber,83,4
65,66,European Style Spring Mix,123,4
88,89,Yogurt Fruit Dip Sliced Apples,123,4
98,99,Local Living Butter Lettuce,83,4
119,120,Cauliflower Florettes,123,4
141,142,Arugula Salad,123,4
142,143,Organic Lemons,24,4
147,148,Nectarines,24,4


In [56]:
products_id = df_fruits['product_id'].values
len(products_id)

1566

In [57]:
aisle_orders = df_allOrdersProducts[df_allOrdersProducts['product_id'].isin(products_id)]
aisle_ordersproducts = aisle_orders.drop(['add_to_cart_order', 'reordered', 'index'], axis=1)
aisle_ordersproducts

,order_id,product_id
1,2,28985
5,2,17794
8,2,43668
11,3,17704
12,3,21903
14,3,46667
30,5,13176
33,5,27966
44,5,37011
51,5,47209


In [77]:
aisle_orders = aisle_ordersproducts.groupby(['order_id']).agg({'product_id': lambda x: tuple(x)})
aisle_orders.columns = ['products']
aisle_orders_str = aisle_ordersproducts.groupby(['order_id']).agg({'product_id': lambda x: tuple(map(str, x))})
aisle_orders_str.columns = ['products']

In [61]:
aisle_orders

,products
order_id,
1,"(10246, 49683, 13176, 47209)"
2,"(28985, 17794, 43668)"
3,"(17704, 21903, 46667)"
5,"(13176, 27966, 37011, 47209, 38693, 48825)"
9,"(21405, 47890, 11182, 14992)"
10,"(24852, 4796, 47766, 4605, 21137, 14992, 3464,..."
12,"(43772,)"
14,"(45066,)"
18,"(34969, 5212, 46969, 27966, 26648, 48745, 2333..."


In [67]:
number_orders = len(aisle_orders)
number_orders

2447934

In [94]:
list_orders = aisle_orders['products'].values
list_orders = list(map(set, list_orders))

In [62]:
aisle_products = aisle_ordersproducts.groupby(['product_id']).count()
aisle_products.columns = ['occurences']

In [65]:
aisle_products = aisle_products.sort_values(by=['occurences'], ascending=False)

In [68]:
support_threshold = 0.001

In [69]:
def frequency(row):
    return row.occurences/number_orders
aisle_products['frequency'] = aisle_products.apply(frequency, axis=1)

In [73]:
frequent_products = aisle_products[aisle_products['frequency']>= support_threshold]


In [92]:
list_frequent_items = [set([item]) for item in frequent_products.index]


In [98]:
def get_supersets(list_frequent_items, list_orders):
    frequent_supersets = []
    for i, element1 in enumerate(list_frequent_items[:-1]):
        for element2 in list_frequent_items[i+1:]:
            candidate = element1.union(element2)
            if len([order for order in list_orders if all(elem in order for elem in candidate)])/number_orders >= support_threshold:
                frequent_supersets.append(candidate)
pairs = get_supersets(list_frequent_items, list_orders)

KeyboardInterrupt: 

In [78]:
dummy_orderproducts = aisle_orders_str['products'].str.join(sep='*').str.get_dummies(sep='*')

MemoryError: 

In [87]:
(1,2).isin((1,2,3))


AttributeError: 'tuple' object has no attribute 'isin'

In [96]:
set((1,2)).union(set((2,3)))

{1, 2, 3}